# Missions Editor

Generate and edit route deck missions, then save back to `board.json`.


In [1]:
import json
from pathlib import Path

import ipywidgets as widgets
from IPython.display import display

board = json.loads(Path("board.json").read_text())
places = {p["id"]: p for p in board.get("places", [])}
roads = board.get("roads", [])


In [3]:
# Read current missions (standard and highways)
route_decks = board.get("routeDecks", {})
place_name = {pid: p.get("name", str(pid)) for pid, p in places.items()}

def format_mission(mission, idx=None):
    ids = mission.get("placeIds", [])
    if len(ids) != 2:
        label = "(invalid)"
    else:
        a, b = ids
        label = f"{place_name.get(a, a)} — {place_name.get(b, b)}"
    score = mission.get("score", "?")
    prefix = f"{idx:02d}. " if idx is not None else ""
    return f"{prefix}{label} (score: {score})"

for deck_name in ["standard", "highways"]:
    missions = route_decks.get(deck_name, [])
    print(f"{deck_name.upper()} MISSIONS (current)")
    if not missions:
        print("(none)")
        continue
    for idx, mission in enumerate(missions, 1):
        print(format_mission(mission, idx))


# Delete all missions
delete_all_missions = widgets.Button(description="Delete All Missions", button_style="danger")

def handle_delete_all(_):
    board["routeDecks"]["standard"] = []
    board["routeDecks"]["highways"] = []
    print("Deleted all missions (in-memory). Save to persist.")

delete_all_missions.on_click(handle_delete_all)
display(delete_all_missions)


STANDARD MISSIONS (current)
(none)
HIGHWAYS MISSIONS (current)
(none)


Button(button_style='danger', description='Delete All Missions', style=ButtonStyle())

In [ ]:
# Redesign missions: short standard, long highways (difficulty-aware)
import random
import heapq
import ipywidgets as widgets
from IPython.display import display

random.seed(7)

# Build weighted graph with tunnel cost doubling
weighted_adj = {pid: [] for pid in places}
for r in roads:
    ids = r.get("placeIds", [])
    if len(ids) != 2:
        continue
    a, b = ids
    w = int(r.get("spaceAmount", 0) or 0)
    if r.get("tunnelCostDouble"):
        w *= 2
    weighted_adj[a].append((b, w))
    weighted_adj[b].append((a, w))


def shortest_path_cost_and_hops(src, dst):
    # Dijkstra to get cost and hops count
    pq = [(0, 0, src)]  # cost, hops, node
    seen = {}
    while pq:
        cost, hops, node = heapq.heappop(pq)
        if node in seen:
            continue
        seen[node] = (cost, hops)
        if node == dst:
            return cost, hops
        for nxt, w in weighted_adj.get(node, []):
            if nxt in seen:
                continue
            heapq.heappush(pq, (cost + w, hops + 1, nxt))
    return None, None


def pick_pairs_by_hops(hops_min, hops_max, target_count, prefer_hops):
    pairs = []
    ids = list(places.keys())
    # Precompute candidate pairs
    candidates = []
    for i in range(len(ids)):
        for j in range(i + 1, len(ids)):
            a, b = ids[i], ids[j]
            cost, hops = shortest_path_cost_and_hops(a, b)
            if cost is None:
                continue
            if hops_min <= hops <= hops_max:
                candidates.append((a, b, cost, hops))
    if not candidates:
        return []

    # Weight by closeness to preferred hop count
    def weight(h):
        return 1.0 / (1 + abs(h - prefer_hops))

    random.shuffle(candidates)
    while candidates and len(pairs) < target_count:
        weights = [weight(c[3]) for c in candidates]
        pick = random.choices(candidates, weights=weights, k=1)[0]
        pairs.append(pick)
        candidates.remove(pick)

    return pairs


def score_from_cost(cost, min_score, max_score):
    # Map cost to score range
    # Clamp cost to a reasonable range
    cost = max(1, min(cost, 40))
    # Normalize 1..40 to 0..1
    t = (cost - 1) / 39.0
    score = min_score + t * (max_score - min_score)
    return int(round(score))


def generate_missions():
    # Standard: 2-5 hops, prefer 3-4, score 7-14
    standard_pairs = pick_pairs_by_hops(2, 5, int(standard_count.value), prefer_hops=3.5)
    # Highways: 5-9 hops, prefer 7, score 15-28
    highway_pairs = pick_pairs_by_hops(5, 9, int(highway_count.value), prefer_hops=7)

    standard = []
    for a, b, cost, hops in standard_pairs:
        standard.append({
            "placeIds": [a, b],
            "score": score_from_cost(cost, 7, 14),
            "hops": hops,
        })

    highways = []
    for a, b, cost, hops in highway_pairs:
        highways.append({
            "placeIds": [a, b],
            "score": score_from_cost(cost, 15, 28),
            "hops": hops,
        })

    board.setdefault("routeDecks", {})
    existing_standard = board["routeDecks"].get("standard", [])
    existing_highways = board["routeDecks"].get("highways", [])
    existing_standard.extend(standard)
    existing_highways.extend(highways)
    board["routeDecks"]["standard"] = existing_standard
    board["routeDecks"]["highways"] = existing_highways


def print_sample(deck_name, limit=10):
    missions = board.get("routeDecks", {}).get(deck_name, [])
    place_name = {pid: p.get("name", str(pid)) for pid, p in places.items()}
    print(f"{deck_name.upper()} MISSIONS (sample)")
    for i, m in enumerate(missions[:limit], 1):
        a, b = m.get("placeIds", [None, None])
        print(f"{i:02d}. {place_name.get(a, a)} — {place_name.get(b, b)} (score: {m.get('score')}, hops: {m.get('hops')})")


standard_count = widgets.IntSlider(description="Standard", min=5, max=60, value=30)
highway_count = widgets.IntSlider(description="Highways", min=5, max=60, value=20)

btn_generate = widgets.Button(description="Generate Missions", button_style="primary")
btn_save = widgets.Button(description="Save Missions to board.json", button_style="success")
output = widgets.Output()


def handle_generate(_):
    generate_missions()
    with output:
        output.clear_output()


def handle_save(_):
    Path("board.json").write_text(json.dumps(board, indent=2))
    with output:
        print("Saved redesigned missions to board.json")

btn_generate.on_click(handle_generate)
btn_save.on_click(handle_save)

display(widgets.HBox([standard_count, highway_count, btn_generate, btn_save]), output)


Output()

In [7]:
# Mission editor (interactive)
import ipywidgets as widgets
from IPython.display import display

# Ensure weighted graph helpers exist
if "weighted_adj" not in globals():
    weighted_adj = {pid: [] for pid in places}
    for r in roads:
        ids = r.get("placeIds", [])
        if len(ids) != 2:
            continue
        a, b = ids
        w = int(r.get("spaceAmount", 0) or 0)
        if r.get("tunnelCostDouble"):
            w *= 2
        weighted_adj[a].append((b, w))
        weighted_adj[b].append((a, w))

if "shortest_path_cost_and_hops" not in globals():
    import heapq

    def shortest_path_cost_and_hops(src, dst):
        pq = [(0, 0, src)]
        seen = {}
        while pq:
            cost, hops, node = heapq.heappop(pq)
            if node in seen:
                continue
            seen[node] = (cost, hops)
            if node == dst:
                return cost, hops
            for nxt, w in weighted_adj.get(node, []):
                if nxt in seen:
                    continue
                heapq.heappush(pq, (cost + w, hops + 1, nxt))
        return None, None

route_decks = board.setdefault("routeDecks", {"standard": [], "highways": []})

place_name = {pid: p.get("name", str(pid)) for pid, p in places.items()}
place_options = sorted([(p.get("name", str(pid)), pid) for pid, p in places.items()], key=lambda x: x[0])


def format_mission(mission, idx=None):
    ids = mission.get("placeIds", [])
    if len(ids) != 2:
        label = "(invalid)"
    else:
        a, b = ids
        label = f"{place_name.get(a, a)} — {place_name.get(b, b)}"
    score = mission.get("score", "?")
    prefix = f"{idx:02d}. " if idx is not None else ""
    return f"{prefix}{label} (score: {score})"


def deck_options(deck_name):
    missions = route_decks.get(deck_name, [])
    return [(format_mission(m, i + 1), i) for i, m in enumerate(missions)]


# UI widgets
deck_select = widgets.ToggleButtons(options=["standard", "highways"], value="standard", description="Deck")
mission_select = widgets.Select(options=deck_options("standard"), description="Missions", rows=8)

city_a = widgets.Dropdown(options=place_options, description="City A")
city_b = widgets.Dropdown(options=place_options, description="City B")
score = widgets.IntSlider(description="Score", min=5, max=30, value=10)

hops_label = widgets.HTML(value="<b>Hops:</b> -")
cost_label = widgets.HTML(value="<b>Min Cost:</b> -")

add_btn = widgets.Button(description="Add New", button_style="primary")
update_btn = widgets.Button(description="Update Selected", button_style="info")
remove_btn = widgets.Button(description="Remove Selected", button_style="danger")
output = widgets.Output()


def refresh_mission_list():
    mission_select.options = deck_options(deck_select.value)


def update_metrics():
    a = city_a.value
    b = city_b.value
    if a == b:
        hops_label.value = "<b>Hops:</b> 0"
        cost_label.value = "<b>Min Cost:</b> 0"
        return
    cost, hops = shortest_path_cost_and_hops(a, b)
    hops_label.value = f"<b>Hops:</b> {hops if hops is not None else 'N/A'}"
    cost_label.value = f"<b>Min Cost:</b> {cost if cost is not None else 'N/A'}"


def handle_deck_change(change):
    refresh_mission_list()


def handle_mission_select(change):
    if mission_select.value is None:
        return
    idx = mission_select.value
    missions = route_decks.get(deck_select.value, [])
    if idx < 0 or idx >= len(missions):
        return
    m = missions[idx]
    ids = m.get("placeIds", [])
    if len(ids) == 2:
        city_a.value = ids[0]
        city_b.value = ids[1]
    score.value = int(m.get("score", score.value) or score.value)
    update_metrics()


def handle_add(_):
    missions = route_decks.get(deck_select.value, [])
    missions.append({"placeIds": [city_a.value, city_b.value], "score": int(score.value)})
    route_decks[deck_select.value] = missions
    refresh_mission_list()


def handle_update(_):
    idx = mission_select.value
    if idx is None:
        return
    missions = route_decks.get(deck_select.value, [])
    if idx < 0 or idx >= len(missions):
        return
    missions[idx] = {"placeIds": [city_a.value, city_b.value], "score": int(score.value)}
    route_decks[deck_select.value] = missions
    refresh_mission_list()


def handle_remove(_):
    idx = mission_select.value
    if idx is None:
        return
    missions = route_decks.get(deck_select.value, [])
    if idx < 0 or idx >= len(missions):
        return
    missions.pop(idx)
    route_decks[deck_select.value] = missions
    refresh_mission_list()



deck_select.observe(handle_deck_change, names="value")
mission_select.observe(handle_mission_select, names="value")
city_a.observe(lambda change: update_metrics(), names="value")
city_b.observe(lambda change: update_metrics(), names="value")

add_btn.on_click(handle_add)
update_btn.on_click(handle_update)
remove_btn.on_click(handle_remove)

refresh_mission_list()
update_metrics()

ui = widgets.VBox([
    deck_select,
    mission_select,
    widgets.HBox([city_a, city_b]),
    widgets.HBox([score, hops_label, cost_label]),
    widgets.HBox([add_btn, update_btn, remove_btn]),
])

display(ui, output)


Output()

In [ ]:
# Deduplicate missions and save
import ipywidgets as widgets
from IPython.display import display

remove_dupes = widgets.Button(description="Remove Duplicate Missions", button_style="warning")
save_all = widgets.Button(description="Save Missions to board.json", button_style="success")
output_check = widgets.Output()


def normalize_key(m):
    ids = m.get("placeIds", [])
    if len(ids) != 2:
        return None
    return tuple(sorted(ids))


def handle_remove_dupes(_):
    for deck in ["standard", "highways"]:
        missions = board.get("routeDecks", {}).get(deck, [])
        seen = set()
        unique = []
        for m in missions:
            key = normalize_key(m)
            if key is None:
                continue
            if key in seen:
                continue
            seen.add(key)
            unique.append(m)
        board["routeDecks"][deck] = unique
    with output_check:
        output_check.clear_output()
        print("Removed duplicate missions.")


def handle_save_all(_):
    Path("board.json").write_text(json.dumps(board, indent=2))
    with output_check:
        print("Saved missions to board.json")

remove_dupes.on_click(handle_remove_dupes)
save_all.on_click(handle_save_all)

display(widgets.HBox([remove_dupes, save_all]), output_check)


Output()